# U-Net Depth (Projeto de dissertação)

In [ ]:
# Clone o repositório da dissertação
!git clone "https://github.com/duraes-antonio/unet_depth"

In [ ]:
import os

# Navegue para o repositório
!cd "unet_depth"
os.chdir("unet_depth")

## Dataset

### Baixar dataset

In [ ]:
!git clone "https://gitlab.com/siddinc/new_depth.git" "./data"

### Particionar (em treino e teste) e instanciar geradores

In [ ]:
from domain.models.data.data_generator import NyuV2Generator
from infra.util.dataset import load_nyu_train_paths

train_path = "./data/nyu2_train.csv"
test_path = "./data/nyu2_test.csv"
batch_size = 4
init_lr = 0.0001

# Defina a semente usada em operações pseudo-aleatórias (como embaralhamento do dataset)
seed = 42

partition, y_path_by_x_path = load_nyu_train_paths(train_path, 0.3, seed)

training_generator = NyuV2Generator(
    path_list=partition['train'], labels=y_path_by_x_path, batch_size=batch_size,
    seed=seed
)
validation_generator = NyuV2Generator(
    path_list=partition['validation'], labels=y_path_by_x_path, batch_size=batch_size,
    seed=seed
)

## Instanciar modelo

### Instanciar serviços para persistência de resultados e blob

In [1]:
from infra.services.model_storage_service_google_drive import ModelStorageServiceGoogleDrive
from domain.services.model_storage_service import ModelStorageService
from infra.services.blob_storage.google_drive_blob_storage_service import GoogleDriveBlobStorageService
from infra.services.test_case_execution_service_mongodb import TestCaseExecutionServiceMongoDB
from infra.services.test_case_service_mongodb import TestCaseServiceMongoDB
from domain.services.test_case_execution_service import TestCaseExecutionService
from domain.services.blob_storage_service import BlobStorageService
from domain.services.test_case_service import TestCaseService

DB_NAME = 'unet_depth'

test_case_serv: TestCaseService = TestCaseServiceMongoDB(DB_NAME)
execution_serv: TestCaseExecutionService = TestCaseExecutionServiceMongoDB(DB_NAME)

blob_service: BlobStorageService = GoogleDriveBlobStorageService()
model_storage: ModelStorageService = ModelStorageServiceGoogleDrive(blob_service)

### Buscar caso de teste para executar e modelo salvo previamente

In [2]:
from domain.models.test_case import TestCaseState

last_test_case = test_case_serv.get_first_available()

# Se não tiver nenhum caso disponível, finalize a execução
if last_test_case is None:
    print('Todos casos de teste finalizados!')
    exit(0)
    raise InterruptedError()

test_case_serv.update_state(last_test_case['id'], TestCaseState.Busy)

# Buscar última execução do caso de teste
last_execution = execution_serv.get_last_execution(last_test_case['id'])

# Buscar último blob do modelo atualizado
if last_execution:
    model_id = last_execution['model_id']

    if model_id:
        model_storage.recover(model_id)

In [5]:
from infra.util.output import print_test_case

print_test_case(last_test_case)

ImportError: cannot import name 'print_test_case' from 'infra.util.output' (C:\Users\AEVO\Desktop\pessoal\IFES\dissertacao\unet_depth\infra\util\output.py)

In [ ]:
from keras import Model
from typing import Optional, Tuple
from domain.models.network import Networks
from domain.models.test_case import TestCase
from keras_unet_collection import models


# Instanciar modelo com base no caso de teste
def get_model(test_case: TestCase, input_shape: Tuple[int, int, int]) -> Optional[Model]:
    backbone = (test_case['backbone']).value
    use_imagenet_weights = test_case['use_imagenet_weights']
    network = test_case['network']

    n_labels = 1
    filter_num = [32, 64, 128, 256, 512]
    activation = 'ReLU'
    out_activation = 'Sigmoid'
    weights = 'imagenet' if use_imagenet_weights else None
    pool = False
    unpool = True
    batch_norm = True

    if network == Networks.UNet:
        return models.unet_2d(
            input_shape, filter_num=filter_num, n_labels=n_labels, stack_num_down=2,
            stack_num_up=2, activation=activation, output_activation=out_activation,
            batch_norm=batch_norm, pool=pool, unpool=unpool, backbone=backbone,
            weights=weights, freeze_backbone=True, freeze_batch_norm=True,
        )

    if network == Networks.AttentionUNet:
        return models.att_unet_2d(
            input_shape, filter_num=filter_num, n_labels=n_labels, stack_num_down=2,
            stack_num_up=2, activation=activation, atten_activation='ReLU', attention='add',
            output_activation=out_activation, batch_norm=batch_norm, pool=pool,
            unpool=unpool, backbone=backbone, weights=weights, freeze_backbone=True,
            freeze_batch_norm=True, name='attunet'
        )
    #
    # if network == Networks.SwinUNet:
    #     return models.swin_unet_2d(
    #         input_size=input_shape, filter_num_begin, n_labels=n_labels, depth=4,
    #         stack_num_down=2, stack_num_up=2, patch_size, num_heads,
    #         window_size, num_mlp, output_activation=out_activation, shift_window=True,
    #         name='swin_unet'
    #     )

    if network == Networks.TransUNet:
        return models.transunet_2d(
            input_size=input_shape, filter_num=filter_num, n_labels=n_labels,
            stack_num_down=2, stack_num_up=2, embed_dim=768, num_mlp=3072,
            num_heads=12, num_transformer=12, activation=activation, mlp_activation='GELU',
            output_activation=out_activation, batch_norm=batch_norm, pool=pool,
            unpool=unpool, backbone=backbone, weights=weights, freeze_backbone=True,
            freeze_batch_norm=True, name='transunet'
        )

    raise ValueError(f"Invalid network '{network}'")

## Preparar modelo

### Definir métricas e função de perda customizadas

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K


def poly_decay(epoch):
    max_epochs = epochs
    base_lr = init_lr
    power = 1.0
    return base_lr * (1 - (epoch / float(max_epochs))) ** power


def depth_loss(y_true, y_pred):
    w1, w2, w3 = 1.0, 1.0, 0.1

    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

    dy_true, dx_true = tf.image.image_gradients(y_true)
    dy_pred, dx_pred = tf.image.image_gradients(y_pred)
    l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)

    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, 1.0)) * 0.5, 0, 1)

    return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))


def depth_acc(y_true, y_pred):
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

### Definir callbacks

In [ ]:
network = last_test_case['network'].value
optimizer: str = last_test_case['optimizer'].value
backbone = str(last_test_case['backbone'].value).lower()
use_image_net = int(last_test_case['use_imagenet_weights'])
epoch = last_execution['last_epoch'] + 1 if last_execution else 1

# Exemplo de formato: 'attention-unet_epoch-15_adam_resnet-101_imagenet-0'
model_name = f'{network}_epoch-{epoch}_{optimizer}_{backbone}_imagenet-{use_image_net}'
csv_log_path = f'{model_name}.log'

project_directory = 'unet_depth'

In [ ]:
from infra.keras.custom_callbacks import CSVResultsSave, TrainedModelSave
from datetime import datetime
from tensorflow.python.keras.callbacks import LearningRateScheduler, EarlyStopping, TensorBoard, CSVLogger

tensorboard_log_path = "logs/fit/"
tensorboard_current_log_path = tensorboard_log_path + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

callbacks = [
    LearningRateScheduler(poly_decay),
    TensorBoard(log_dir=tensorboard_current_log_path, histogram_freq=1),
    EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True),
    CSVLogger(csv_log_path),
    CSVResultsSave(blob_service, csv_log_path),
    TrainedModelSave(model_storage, model_name)
]

## Compilar e treinar

In [ ]:
input_shape = (640, 480, 3)
width, height, = input_shape
unet_model = get_model(last_test_case, input_shape)
epochs = 50

unet_model.compile(loss=depth_loss, metrics=[depth_acc], optimizer=optimizer)

In [ ]:
unet_model.fit(training_generator, validation_data=validation_generator, callbacks=callbacks, epochs=epochs)

In [ ]:
% tensorboard --logdir $tensorboard_log_path

## Avaliar na base de teste

In [ ]:
from infra.util.dataset import read_nyu_csv

img_path_pairs = read_nyu_csv(test_path)
labels = {x_path: y_path for x_path, y_path in img_path_pairs}
test_paths = [x_path for x_path, y_path in img_path_pairs]
partition = {'test': test_paths}

In [ ]:
from infra.util.preprocessing import preprocess_image
from infra.util.preprocessing import preprocess_depth_map
import numpy

x_test = numpy.empty((len(test_paths), height, width, 3))
y_test = numpy.empty((len(test_paths), height, width, 1))

for index, ID in enumerate(partition['test'][:]):
    x_test[index,] = preprocess_image(ID)
    y_test[index,] = preprocess_depth_map(labels[ID])

In [ ]:
unet_model.evaluate(x_test, y_test)

### Plot de imagens e predições de exemplo

In [ ]:
import matplotlib.pyplot as plot

predicted = unet_model.predict(x_test)

for index in range(len(test_paths) - 600):
    # Predição
    prediction = predicted[index]
    prediction = numpy.squeeze(prediction, axis=-1)
    plot.subplot(1, 3, 1)
    plot.axis("off")
    plot.imshow(prediction, cmap=plot.get_cmap('inferno_r'))

    # Ground truth
    path = partition['test'][index]
    label_path = labels[path]
    plot.subplot(1, 3, 2)
    plot.axis("off")
    target_depth_map = preprocess_depth_map(label_path)
    target_depth_map = numpy.squeeze(target_depth_map, axis=-1)
    plot.imshow(target_depth_map, cmap=plot.get_cmap('inferno_r'))

    # Imagem original
    plot.subplot(1, 3, 3)
    plot.axis("off")
    original_image = preprocess_image(path)
    plot.imshow(original_image)

    plot.show()